In [1]:
import numpy as np
from sklearn.cluster import DBSCAN, KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score

# Загрузка данных

Два `.csv` находятся в корне архива. Кроме того после обработки фотографий можно подать на вход одноимённые файлы из папки `output`.

In [2]:
dists = np.genfromtxt('./dists.csv', delimiter=',')
labels = np.genfromtxt('./labels.csv', delimiter=',').astype(int)

# Поиск наиболее похожих объектов

Применяем готовый алгоритм поиска ближайших соседей с евклидовой метрикой и сохраняем результат.

In [3]:
nbrs = NearestNeighbors(n_neighbors=4).fit(dists)
distances, indices = nbrs.kneighbors(dists)

In [4]:
np.savetxt('./output/neighbors.csv', labels[indices], delimiter=',')

# Кластеризация ладоней и поиск числа уникальных людей

Избавляемся от выбросов при помощи DBScan, подбираем при помощи коэффициента силуэта оптимальное число кластеров и сохраняем результат.

In [5]:
dbscan = DBSCAN(eps=70).fit_predict(dists)

score = []

for i in range(5, 50):
    kmeans = KMeans(n_clusters=i).fit_predict(dists[dbscan != -1])
    score.append(silhouette_score(dists[dbscan != -1], kmeans))

best_clust = list(range(10, 40))[np.argmax(score)]

kmeans = KMeans(n_clusters=best_clust).fit_predict(dists[dbscan != -1])

print('Best n_clusters: ', best_clust)

Best n_clusters:  14


Теперь приведём даннные к нужному формату (то есть к таблице вида «Персона № – имена изображений ладоней»).

In [6]:
clust_labels = np.hstack([labels[dbscan != -1], labels[dbscan == -1]])
clusters = np.hstack([kmeans, dbscan[dbscan == -1]])

clust_res = []
lens = []

for i in range(-1, best_clust):
    clust_res.append(clust_labels[clusters == i])
    lens.append(len(clust_labels[clusters == i]))

width = np.max(lens)

In [7]:
persons = np.zeros(width + 1)

for i, person in enumerate(clust_res):
    temp = [i - 1] + list(person) + [0] * (width - len(person))
    persons = np.vstack([persons, np.array(temp)])

persons = persons[1:]

In [8]:
np.savetxt('./output/persons.csv', persons, delimiter=',')